In [1]:
# importing tools 
import s3fs
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt 
import cmocean
import math
from matplotlib.patches import Ellipse
#for removing warnings
import warnings

In [2]:
# open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path1 = 's3://data/CREG12.L75-REF08_mesh_zgr.zarr'
data_path2 = 's3://data/vel_dataIII.zarr/vel_dataIII.zarr'
data_path3 = 's3://data/lat_lon_scalar.zarr'

remote_files1 = fsg.glob(data_path1)
remote_files2 = fsg.glob(data_path2)
remote_files3 = fsg.glob(data_path3)

store1 = s3fs.S3Map(root=data_path1, s3=fsg, check=False)
store2 = s3fs.S3Map(root=data_path2, s3=fsg, check=False)
store3 = s3fs.S3Map(root=data_path3, s3=fsg, check=False)

dzz = xr.open_zarr(store=store1, consolidated=True)
dv = xr.open_zarr(store=store2, consolidated=True)
dl = xr.open_zarr(store=store3, consolidated=True)

In [3]:
bathym = dzz.mbathy.isel(x=slice(200,1400),y=slice(650,1800)).squeeze(axis=0)

In [4]:
def open_s3fs(path):
    # open the data with s3fs from the databucket 
    fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
    data_path = f's3://data/{path}'
    remote_files = fsg.glob(data_path)
    fileset = [fsg.open(file) for file in remote_files]
    #open the dataset 
    dset = xr.open_mfdataset(fileset, 
                             #combine='by_coords',
                             compat='override')
    return dset

In [6]:
d15 = open_s3fs('mooring_barents/A-TWAIN_2015-2017_AT200-3_22-118m_ACDP_16640.nc')
d13 = open_s3fs('mooring_barents/A-TWAIN_2013-2015_AT200-2_14-121m_ACDP_16640.nc')
d17 = open_s3fs('mooring_barents/A-TWAIN_2017-2019_AT200-4_20-167m_ACDP_16640.nc')

y1 = open_s3fs('mooring_yermark/Y1.nc')
y2 = open_s3fs('mooring_yermark/Y2.nc')
y3 = open_s3fs('mooring_yermark/Y3.nc')

MB = open_s3fs('mooring_lofoten/PROVOLO_Mooring_MB.nc')
MW = open_s3fs('mooring_lofoten/PROVOLO_Mooring_MW.nc')
MN = open_s3fs('mooring_lofoten/PROVOLO_Mooring_MN.nc')
MS = open_s3fs('mooring_lofoten/PROVOLO_Mooring_MS.nc')

ANO18 = open_s3fs('moorings_/AON_Sig_ADCP_BS3_2016_2018.nc')
ANO16 = open_s3fs('moorings_/AON_LR_ADCP_BS3_2014_2016.nc')
ANO14 = open_s3fs('moorings_/AON_LR_ADCP_BS3_2013_2014.nc')

E3 = open_s3fs('mooring_nord_svalbard/NL_E3.nc')
W3 = open_s3fs('mooring_nord_svalbard/NL_W3.nc')
E2 = open_s3fs('mooring_nord_svalbard/NL_E2.nc') 
W2 = open_s3fs('mooring_nord_svalbard/NL_W2.nc')
E1 = open_s3fs('mooring_nord_svalbard/NL_E1.nc') 
W1 = open_s3fs('mooring_nord_svalbard/NL_W1.nc') 

In [38]:
latlon = [(31.13247,81.24202),
          (31.13533,81.24255),
          (31.14506,81.24587)
          ,(011.1189, 69.5289),
          (013.16845,68.58759),
          (013.19866,68.56109),
          (012.45082,68.50128),
          (5.48733,80.03876),
          (5.56333,79.44093),
          (24.00000,81.24925),
          (18.29052,81.10979),
          (23.59853,81.30813),
          (18.23789,81.22686),
          (23.59982,81.35453),
          (18.23730,81.27356)]
moor_l = [(836, 440),
            (838, 440),
            (836, 439),
            (923, 157), 
            (930, 149),
            (932, 147),
            (929, 142),
            (774, 344+6),
            (766, 358+6),
            (773, 346+6),
            (334, 978),
            (334, 978),
            (334, 978),
            (817, 425), # new moorings from here
            (800, 405),
            (810, 425),
            (802, 413),
            (810, 427),
            (797, 414)]
moor_vec = [np.array([[ 0.27541674, -0.96132493],
        [-0.96132493, -0.27541674]]),
         np.array([[ 0.1863939, -0.9824751],
        [-0.9824751, -0.1863939]]),
         np.array([[ 0.42605245, -0.90469846],
        [-0.90469846, -0.42605245]]),
         np.array([[ 0.86834446, 0.49596158],
        [-0.49596158,  0.86834446]]),
         np.array([[-0.58294305, -0.81251301],
        [-0.81251301,  0.58294305]]),
         np.array([[ 0.86822067, -0.49617826],
        [ 0.49617826,  0.86822067]]),
         np.array([[ 0.76994574, -0.63810937],
        [ 0.63810937,  0.76994574]]),
         np.array([[ 0.62741291, -0.77868674],
        [-0.77868674, -0.62741291]]),
         np.array([[-0.60092325, -0.79930673],
        [-0.79930673,  0.60092325]]),
         np.array([[ 0.62481889, -0.78076972],
        [-0.78076972, -0.62481889]]),
         np.array([[ 0.76100801,  0.64874249],
        [-0.64874249,  0.76100801]]),
         np.array([[ 0.80047202,  0.59937012],
        [-0.59937012,  0.80047202]]),
         np.array([[ 0.89578352,  0.44449059],
        [-0.44449059,  0.89578352]]),
         np.array([[ 0.9975322 , -0.07021048],
        [ 0.07021048,  0.9975322 ]]),
         np.array([[ 0.9732993 , -0.22953971],
        [ 0.22953971,  0.9732993 ]]),
        np.array([[ 0.99975175, -0.0222807 ],
        [ 0.0222807 ,  0.99975175]]),
        np.array([[ 0.84114404, -0.54081116],
        [ 0.54081116,  0.84114404]]),
        np.array([[ 0.99493596,  0.10051086],
        [-0.10051086,  0.99493596]]),
         np.array([[ 0.98863153, -0.15035855],
        [ 0.15035855,  0.98863153]])]

c_list = ['gold', 'orangered', 'g',
          'r', 'm', 'y','tab:orange',
          'tab:pink', 'limegreen', 'maroon', 
          'dodgerblue', 'c', 'peru','deeppink','darkviolet','forestgreen','bisque','coral','palegreen']


In [8]:
# Barents sea
u13 = d13.UCUR.squeeze(axis=0)
v13 = d13.VCUR.squeeze(axis=0)
u15 = d15.UCUR.squeeze(axis=0)
v15 = d15.VCUR.squeeze(axis=0)
u17 = d17.UCUR.squeeze(axis=0)
v17 = d17.VCUR.squeeze(axis=0)

# Lofoten
ub = MB.u
vb = MB.v
uw = MW.u
vw = MW.v
un = MN.u
vn = MN.v
us = MS.u
vs = MS.v

# Yermark
u1 = y1.u
v1 = y1.v
u2 = y2.u
v2 = y2.v
u3 = y3.u
v3 = y3.v

# Beaufort
'sliter med å finne ut hvordan få gode oversiktlige'
u14 = ANO14.UCUR
v14 = ANO14.VCUR
u16 = ANO16.UCUR
v16 = ANO16.VCUR
u18 = ANO18.UCUR
v18 = ANO18.VCUR

# Nord Svalbard, DEPTH & TIME
ue1 = E1.UCUR
ve1 = E1.VCUR
uw1 = W1.UCUR
vw1 = W1.VCUR
ue2 = E2.UCUR
ve2 = E2.VCUR
uw2 = W2.UCUR
vw2 = W2.VCUR
ue3 = E3.UCUR
ve3 = E3.VCUR
uw3 = W3.UCUR
vw3 = W3.VCUR

In [ ]:
def tot_var_nDEPTH(u,v):
    var = np.zeros(len(u.nDEPTH))
    for i in range(len(u.nDEPTH)):
        u = u.isel(nDEPTH=i)
        nu = np.isnan(u)
        u_ = u[~nu]        
        v = v.isel(nDEPTH=i)
        nv = np.isnan(v)
        v_ = u[~nv]
        var[i] = np.sqrt(np.var(u_.values) + np.var(v_.values))
    return var

def tot_var_DEPTH(u,v):
    var = np.zeros(len(u.DEPTH))
    for i in range(len(u.DEPTH)):
        u = u.isel(DEPTH=i)
        nu = np.isnan(u)
        u_ = u[~nu]        
        v = v.isel(DEPTH=i)
        nv = np.isnan(v)
        v_ = u[~nv]
        var[i] = np.sqrt(np.var(u_.values) + np.var(v_.values))
    return var

def tot_var_BINDEPTH(u,v):
    var = np.zeros(len(u.BINDEPTH))
    for i in range(len(u.BINDEPTH)):
        u = u.isel(BINDEPTH=i)
        nu = np.isnan(u)
        u_ = u[~nu]        
        v = v.isel(BINDEPTH=i)
        nv = np.isnan(v)
        v_ = u[~nv]
        var[i] = np.sqrt(np.var(u_.values) + np.var(v_.values))
    return var

In [25]:
a = ue1.isel(DEPTH=0)
a

<xarray.DataArray 'UCUR' (TIME: 10396)>
dask.array<getitem, shape=(10396,), dtype=float64, chunksize=(10396,), chunktype=numpy.ndarray>
Coordinates:
  * TIME     (TIME) datetime64[ns] 2018-09-16T09:00:00 ... 2019-11-23T12:00:00
    DEPTH    float64 30.0
Attributes:
    standard_name:  eastward_sea_water_velocity
    units:          m s-1
    long_name:      absolute eastward sea water velocity

In [26]:
b = np.isnan(a)
b

<xarray.DataArray 'UCUR' (TIME: 10396)>
dask.array<isnan, shape=(10396,), dtype=bool, chunksize=(10396,), chunktype=numpy.ndarray>
Coordinates:
  * TIME     (TIME) datetime64[ns] 2018-09-16T09:00:00 ... 2019-11-23T12:00:00
    DEPTH    float64 30.0
Attributes:
    standard_name:  eastward_sea_water_velocity
    units:          m s-1
    long_name:      absolute eastward sea water velocity

In [27]:
d = a[~b]
d.values

array([0.26549368, 0.30595732, 0.3228027 , ..., 0.31502329, 0.29097518,
       0.30032608])

In [20]:
d

<xarray.DataArray 'UCUR' (TIME: 6884)>
dask.array<getitem, shape=(6884,), dtype=float64, chunksize=(6884,), chunktype=numpy.ndarray>
Coordinates:
  * TIME     (TIME) datetime64[ns] 2018-09-16T09:00:00 ... 2019-11-09T03:59:5...
    DEPTH    float64 30.0
Attributes:
    standard_name:  eastward_sea_water_velocity
    units:          m s-1
    long_name:      absolute eastward sea water velocity

In [28]:
ue1.values

array([[0.26549368, 0.32335252, 0.33902117, ..., 0.2948563 , 0.28778329,
        0.28071027],
       [0.30595732, 0.35581129, 0.35967802, ..., 0.30207978, 0.29797973,
        0.29387967],
       [0.3228027 , 0.37266681, 0.35006978, ..., 0.27694987, 0.27028674,
        0.26362362],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])